In [1]:
import pandas as pd
import nltk
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support,accuracy_score
from nltk.tokenize import sent_tokenize

from datasets import Dataset, concatenate_datasets
from transformers import (
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    DataCollatorForSeq2Seq,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
)

from typing import List, Tuple
from memotion_utility import load_dataset_text_only,load_data

2024-08-01 12:01:36.266683: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-01 12:01:36.266816: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-01 12:01:36.394021: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
CSV_FILE = '/kaggle/input/memotion-dataset-7k/memotion_dataset_7k/labels.csv'
MAX_LEN = 1024
LR = 2e-5
batch_size = 16
epochs = 50
metric_name = 'f1_macro'

model_name = 'google/flan-t5-small'

downsample = False
captions = '/kaggle/input/memotion-with-captions/caption_BLIP.csv'

In [3]:
dataset = load_dataset_text_only(CSV_FILE, downsample = True,captions = captions,mode='text_only')

train : 
 label
1    1951
0    1951
Name: count, dtype: int64
val : 
 label
1    342
0    217
Name: count, dtype: int64
test : 
 label
1    856
0    542
Name: count, dtype: int64


In [4]:
dataset['train'][0]

{'text': 'Grandparents and technology... TA Geez Grandma! It\'s not that hard! Go into Settings... select wi-fi... Select it! Tap it with your finger... OMG any finger!! Grrrrrrr FWD: FWD: "HA THIS IS TOTALLY ME I HOPE YOU HAVE A GRATE DAY LOVE GRAMMA"',
 'label': 'offensive',
 '__index_level_0__': 5419}

In [5]:
# import os

# import pandas as pd
# from datasets import Dataset


# def load_dataset(add_caption = True) -> Dataset:
#     """Load dataset."""
    
#     dataset_pandas = pd.read_csv(CSV_FILE)[['text_corrected','offensive']]
    
#     dataset_pandas['offensive'] = np.where(dataset_pandas['offensive'] == 'not_offensive', 'not offensive', 'offensive')

    
#     dataset_pandas = dataset_pandas.rename(columns={'offensive': 'label','text_corrected' : 'text'})

#     dataset_pandas["label"] = dataset_pandas["label"].astype(str)
#     dataset_pandas["text"] = dataset_pandas["text"].astype(str)
    
#     dataset = Dataset.from_pandas(dataset_pandas)
#     dataset = dataset.shuffle(seed=42)
#     dataset = dataset.train_test_split(test_size=0.2,seed=42,stratify_by_column='label')

#     return dataset

In [6]:
# def downsample_training_data(dataset: Dataset) -> Dataset:
#     """Downsample the training data to have equal number of offensive and not offensive labels."""
    
#     train_dataset = dataset['train'].to_pandas()
    
#     # Separate the offensive and not offensive samples
#     offensive_samples = train_dataset[train_dataset['label'] == 'offensive']
#     not_offensive_samples = train_dataset[train_dataset['label'] == 'not offensive']
    
#     # Determine the smaller class size
#     min_samples = min(len(offensive_samples), len(not_offensive_samples))
    
#     # Downsample the larger class
#     offensive_samples_downsampled = offensive_samples.sample(min_samples, random_state=42)
#     not_offensive_samples_downsampled = not_offensive_samples.sample(min_samples, random_state=42)
    
#     # Combine the downsampled samples
#     train_dataset_downsampled = pd.concat([offensive_samples_downsampled, not_offensive_samples_downsampled])
    
#     # Shuffle the combined dataset
#     train_dataset_downsampled = train_dataset_downsampled.sample(frac=1, random_state=42).reset_index(drop=True)
    
#     # Convert back to Dataset format
#     train_dataset_downsampled = Dataset.from_pandas(train_dataset_downsampled)
    
#     # Update the train split with the downsampled dataset
#     dataset['train'] = train_dataset_downsampled
    
#     return dataset

In [7]:
# Load the dataset
#dataset = load_dataset()

# Print the number of examples per class for both train and test splits
train_labels = dataset['train']['label']
val_labels = dataset['validation']['label']
test_labels = dataset['test']['label']

train_counts = pd.Series(train_labels).value_counts()
val_counts = pd.Series(val_labels).value_counts()
test_counts = pd.Series(test_labels).value_counts()

print("Number of examples per class in the training set:")
print(train_counts)
print("\nNumber of examples per class in the val set:")
print(val_counts)
print("\nNumber of examples per class in the test set:")
print(test_counts)

Number of examples per class in the training set:
offensive        1951
not offensive    1951
Name: count, dtype: int64

Number of examples per class in the val set:
offensive        342
not offensive    217
Name: count, dtype: int64

Number of examples per class in the test set:
offensive        856
not offensive    542
Name: count, dtype: int64


In [8]:
# # Downsample the training data
# dataset = downsample_training_data(dataset)

# dataset['train'][1]

In [9]:
# train_labels = dataset['train']['label']
# test_labels = dataset['test']['label']

# train_counts = pd.Series(train_labels).value_counts()
# test_counts = pd.Series(test_labels).value_counts()

# print("Number of examples per class in the training set:")
# print(train_counts)
# print("\nNumber of examples per class in the test set:")
# print(test_counts)

In [10]:
# Load tokenizer of FLAN-t5
tokenizer = AutoTokenizer.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

In [11]:
# The maximum total input sequence length after tokenization.
# Sequences longer than this will be truncated, sequences shorter will be padded.

tokenized_inputs = concatenate_datasets([dataset["train"], dataset["validation"],dataset["test"]]).map(
    lambda x: tokenizer(x["text"], truncation=True),
    batched=True,
    remove_columns=["text", "label"],
)

Map:   0%|          | 0/5859 [00:00<?, ? examples/s]

In [12]:
max_source_length = max([len(x) for x in tokenized_inputs["input_ids"]])
print(f"Max source length: {max_source_length}")

Max source length: 321


In [13]:
# The maximum total sequence length for target text after tokenization.
# Sequences longer than this will be truncated, sequences shorter will be padded."
tokenized_targets = concatenate_datasets([dataset["train"], dataset["test"]]).map(
    lambda x: tokenizer(x["label"], truncation=True),
    batched=True,
    remove_columns=["text", "label"],
)
max_target_length = max([len(x) for x in tokenized_targets["input_ids"]])
print(f"Max target length: {max_target_length}")

Map:   0%|          | 0/5300 [00:00<?, ? examples/s]

Max target length: 3


In [14]:
# Define training args
training_args = Seq2SeqTrainingArguments(
    output_dir='./results"',
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    fp16=False,
    seed = 110,
    learning_rate=LR,
    num_train_epochs=epochs,
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    report_to="none",
    generation_max_length = MAX_LEN
)

In [15]:
def preprocess_function(sample: Dataset, padding: str = "max_length") -> dict:
    """Preprocess the dataset."""

    # add prefix to the input for t5
    prefix = 'Is the following text offensive or not offensive ? :'
    inputs = [prefix + item for item in sample["text"]]

    # tokenize inputs
    model_inputs = tokenizer(
        inputs, max_length=max_source_length, padding=padding, truncation=True
    )

    # Tokenize targets with the `text_target` keyword argument
    labels = tokenizer(
        text_target=sample["label"],
        max_length=max_target_length,
        padding=padding,
        truncation=True,
    )

    # If we are padding here, replace all tokenizer.pad_token_id in the labels by -100 when we want to ignore
    # padding in the loss.
    if padding == "max_length":
        labels["input_ids"] = [
            [(la if la != tokenizer.pad_token_id else -100) for la in label]
            for label in labels["input_ids"]
        ]

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [16]:
def postprocess_text(
    preds: List[str], labels: List[str]
) -> Tuple[List[str], List[str]]:
    """helper function to postprocess text"""
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]

    # rougeLSum expects newline after each sentence
    preds = ["\n".join(sent_tokenize(pred)) for pred in preds]
    labels = ["\n".join(sent_tokenize(label)) for label in labels]

    return preds, labels

In [17]:
def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)
    
    #print(decoded_preds)
    
    decoded_preds = [1 if pred == "offensive" else 0 for pred in decoded_preds]
    decoded_labels = [1 if label == "offensive" else 0 for label in decoded_labels]

    # Compute precision, recall, and F1 score for both classes and macro average
    precision, recall, f1, _ = precision_recall_fscore_support(decoded_labels, decoded_preds, average=None, labels=[0, 1])
    precision_macro, recall_macro, f1_macro, _ = precision_recall_fscore_support(decoded_labels, decoded_preds, average='macro')
    acc = accuracy_score(decoded_labels,decoded_preds)
    
    result = {
        "accuracy" : round(acc * 100, 4),
        "precision_not_offensive": round(precision[0] * 100, 4),
        "recall_not_offensive": round(recall[0] * 100, 4),
        "f1_not_offensive": round(f1[0] * 100, 4),
        "precision_offensive": round(precision[1] * 100, 4),
        "recall_offensive": round(recall[1] * 100, 4),
        "f1_offensive": round(f1[1] * 100, 4),
        "precision_macro": round(precision_macro * 100, 4),
        "recall_macro": round(recall_macro * 100, 4),
        "f1_macro": round(f1_macro * 100, 4),
    }
    
    prediction_lens = [
        np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds
    ]
    result["gen_len"] = np.mean(prediction_lens)
    
    return result


In [18]:
def train() -> None:
    """Train and test the model."""

    tokenized_dataset = dataset.map(
        preprocess_function, batched=True, remove_columns=['text','label']
    )
    
    print(f"Keys of tokenized dataset: {list(tokenized_dataset['train'].features)}")

    # load model from the hub
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

    nltk.download("punkt")

    # we want to ignore tokenizer pad token in the loss
    label_pad_token_id = -100
    # Data collator
    data_collator = DataCollatorForSeq2Seq(
        tokenizer,
        model=model,
        label_pad_token_id=label_pad_token_id,
        pad_to_multiple_of=8,
    )

    # Create Trainer instance
    trainer = Seq2SeqTrainer(
        model=model,
        args=training_args,
        data_collator=data_collator,
        train_dataset=tokenized_dataset['train'],
        eval_dataset=tokenized_dataset['validation'],
        compute_metrics=compute_metrics,
    )

    # TRAIN
    trainer.train()
    
    # TEST
    evaluation_results = trainer.evaluate(eval_dataset = tokenized_dataset['test'])
    print(evaluation_results)

In [19]:
train()

Map:   0%|          | 0/3902 [00:00<?, ? examples/s]

Map:   0%|          | 0/559 [00:00<?, ? examples/s]

Map:   0%|          | 0/1398 [00:00<?, ? examples/s]

Keys of tokenized dataset: ['__index_level_0__', 'input_ids', 'attention_mask', 'labels']


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Epoch,Training Loss,Validation Loss,Accuracy,Precision Not Offensive,Recall Not Offensive,F1 Not Offensive,Precision Offensive,Recall Offensive,F1 Offensive,Precision Macro,Recall Macro,F1 Macro,Gen Len
1,No log,0.286153,61.538500,58.333300,3.225800,6.113500,61.608800,98.538000,75.815500,59.971100,50.881900,40.964500,2.021467
2,No log,0.286009,61.001800,40.000000,0.921700,1.801800,61.191300,99.122800,75.669600,50.595700,50.022200,38.735700,2.008945
3,0.406000,0.315942,38.819300,38.532100,96.774200,55.118100,50.000000,2.046800,3.932600,44.266100,49.410500,29.525300,2.974955
4,0.406000,0.284846,59.928400,33.333300,3.225800,5.882400,60.966500,95.906400,74.545500,47.149900,49.566100,40.213900,2.037567
5,0.346500,0.280896,60.822900,33.333300,0.921700,1.793700,61.121200,98.830400,75.530700,47.227200,49.876000,38.662200,2.010733
6,0.346500,0.281486,60.465100,25.000000,0.921700,1.777800,60.980000,98.245600,75.252000,42.990000,49.583600,38.514900,2.014311
7,0.325700,0.280807,60.644000,28.571400,0.921700,1.785700,61.050700,98.538000,75.391500,44.811100,49.729800,38.588600,2.012522
8,0.325700,0.280115,60.644000,28.571400,0.921700,1.785700,61.050700,98.538000,75.391500,44.811100,49.729800,38.588600,2.012522
9,0.319800,0.280251,60.644000,28.571400,0.921700,1.785700,61.050700,98.538000,75.391500,44.811100,49.729800,38.588600,2.012522
10,0.319800,0.298226,47.048300,40.000000,72.811100,51.634000,64.024400,30.701800,41.502000,52.012200,51.756400,46.568000,2.706619


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight'].


{'eval_loss': 0.31550323963165283, 'eval_accuracy': 52.9328, 'eval_precision_not_offensive': 39.2593, 'eval_recall_not_offensive': 39.1144, 'eval_f1_not_offensive': 39.1867, 'eval_precision_offensive': 61.5385, 'eval_recall_offensive': 61.6822, 'eval_f1_offensive': 61.6103, 'eval_precision_macro': 50.3989, 'eval_recall_macro': 50.3983, 'eval_f1_macro': 50.3985, 'eval_gen_len': 2.3862660944206007, 'eval_runtime': 17.8671, 'eval_samples_per_second': 78.245, 'eval_steps_per_second': 9.795, 'epoch': 50.0}
